## Port of Oakland: Inventory Control
Lesson objectives: Use Monte-Carlo/bootstrapping/simulation methods to estimate event probabilities.

In [1]:
import math
import numpy as np
import pandas as pd

## Q1
Write a function to simulate trucks, `trucksim`, which takes as input the following: 
* A number, n, of trucks to simulate
* A dictionary, d, of value-probability pairs which represent the distribution of fill rates. For the case above it would look like { “<10” : .08, “10-15” : .27, ... }
  
The output of this function should be a list n different truck levels, randomly simulated. For example, if n = 1 and d = {“ < 10” : .5, “ > 10” : .5}, then the result will either be [“ < 10”] or [“ > 10”].

In [2]:
def trucksim(n, d):
    """
    n: a number, of trucks to simulate
    d: a dictionary, of value-probability pairs which represent the distribution of fill rates. 
    """
    d_choice = [x for x in d.keys()]
    d_prob = [x for x in d.values()]
    return np.random.choice(d_choice, size=n, replace=True, p=d_prob).tolist()

## Q2
Using the function above, write a wrapper function to bootstrap confidence intervals. In particular, write a function called `truck1CI`, that takes the following arguments:
* A string (“keyname”), the name of a single truck level (“10-15”, “15-20”, ..)
* A simulated list (simlist)
* the number of times to bootstrap (n)
* a, which is equal to 1 - the confidence interval level. So, if we want a 90% Confidence interval, a will be equal to .1.
  
The function should create n samples (or bootstraps) which have the same length as simlist and are created by sampling, with replacement from simlist. In other words, the function create n new samples from the original simlist. The function should calculate the percentage of each of these n lists that are equal to “keyname”. Finally, `truck1C1` should return the mean, 1-a/2 , a/2 percentiles of this percentage and the standard deviation of the bootstrapped means.
  
The last item returned by the function, the standard deviation of the bootstrapped mean, represents the standard error on our estimator (in this case, the sample mean).
  
Please answer the following questions, using a single truck simulation of length 1,000 as input and using the information from table 2.2:

In [3]:
def truck1CI(keyname, simlist, n, a):
    """
    keyname: a string, the name of a single truck level (“10-15”, “15-20”, ..)
    simlist: a simulated list
    n: the number of times to bootstrap
    a: which is equal to 1 - the confidence interval level
    """
    percentage = [np.random.choice(simlist, size=len(simlist), replace=True, p=None).tolist().count(keyname) 
                  / float(len(simlist)) for i in range(n)]
    return {"Confid. Level":str(int((1- a)*100)) + "%",
            "Truck Fill Level":keyname,
            "Number of Bootstraps":n,
            "Est. Value":np.mean(percentage), 
            "CI Lower Bound":np.percentile(percentage, a / 2 *100),
            "CI Upper Bound":np.percentile(percentage, (1 - a / 2) *100),
            "Std. Error":np.std(percentage) / math.sqrt(n)}

In [4]:
#table 2.2
d = {"<10":.08,
     "10-15":.27,
     "15-20":.10,
     "20-25":.11,
     "25-30":.15,
     "30-35":.20,
     "35-37":.07,
     "37+":0.02}

### Q2 (a)
Fill out the following table:

In [5]:
np.random.seed(314)
simlist = trucksim(1000, d)
pd.DataFrame([truck1CI("30-35", simlist, 100, .1),
              truck1CI("30-35", simlist, 1000, .1),
              truck1CI("30-35", simlist, 2500, .1),
              truck1CI("30-35", simlist, 5000, .1),
              truck1CI("<10", simlist, 100, .1),
              truck1CI("<10", simlist, 1000, .1),
              truck1CI("<10", simlist, 2500, .1),
              truck1CI("<10", simlist, 5000, .1)],
              columns = ["Confid. Level", "Truck Fill Level", "Number of Bootstraps",
                         "Est. Value", "CI Lower Bound", "CI Upper Bound", "Std. Error"])

,Confid. Level,Truck Fill Level,Number of Bootstraps,Est. Value,CI Lower Bound,CI Upper Bound,Std. Error
0,90%,30-35,100,0.185490,0.16695,0.20305,0.001094
1,90%,30-35,1000,0.187035,0.16600,0.20700,0.000391
2,90%,30-35,2500,0.186836,0.16700,0.20800,0.000247
3,90%,30-35,5000,0.186943,0.16700,0.20800,0.000174
4,90%,<10,100,0.102060,0.08790,0.12005,0.001020
5,90%,<10,1000,0.101117,0.08500,0.11600,0.000300
6,90%,<10,2500,0.100787,0.08500,0.11700,0.000193
7,90%,<10,5000,0.101058,0.08600,0.11700,0.000133


### Q2 (b)
Did this behave as expected? In particular, comment on the range of the confidence interval and the size of the standard error as the number of bootstraps increases.
  
It behaves as expected. As the number of bootstraps increases, the range of confidence interval becomes more stable and converges to a certain range. The standard error of sample mean decreases as the sample size increases.

## Q3
Using the same process, use the original data (2.2) to build confidence intervals around the percentage of trucks for each type you would expect if 4,989 trucks went out.

In [6]:
simlist = trucksim(4989, d)
pd.DataFrame([truck1CI("35-37", simlist, 200, .05),
              truck1CI("37+", simlist, 200, .05),
              truck1CI("35-37", simlist, 200, .1),
              truck1CI("37+", simlist, 200, .1)],
              columns = ["Confid. Level", "Truck Fill Level", "Number of Bootstraps",
                         "Est. Value", "CI Lower Bound", "CI Upper Bound", "Std. Error"])

,Confid. Level,Truck Fill Level,Number of Bootstraps,Est. Value,CI Lower Bound,CI Upper Bound,Std. Error
0,95%,35-37,200,0.070869,0.063740,0.078382,0.000261
1,95%,37+,200,0.020453,0.017037,0.024659,0.000136
2,90%,35-37,200,0.071338,0.066526,0.076979,0.000232
3,90%,37+,200,0.020160,0.017238,0.023261,0.000136


In [7]:
print float(368) / 4989 #percentage of trucks at level 35-37 in the test of XXX’s algorithm
print float(108) / 4989 #percentage of trucks at level 37+ in the test of XXX’s algorithm

0.0737622770094
0.0216476247745


After completing the table above, what results do you draw about XXX’s algorithm?
  
From this method, we can not say that XXX’s algorithm is better. In the test, 7.376% trucks are at level 35-37 feet and 2.164% are at level 37+, which fall between the confidence intervals we calculated from the simulation using the original data. This means even though the XXX's algorithm is the same as the current one, there is still a good chance that we get the test result. Therefore, we don't have enough evidence to say XXX’s algorithm is better than the current one.

## Q4
Let’s now compute the likelihood that XXX’s algorithm is better than than the current algorithm by using a Monte-Carlo algorithm. To do this repeat the following 1,000 times:
* Using `trucksim`, create a simulated list of 4,989 trucks under the null hypothesis that there is no difference.
* Calculate the number of trucks that that have between 35-37 feet filled. If this is greater than or equal to 368, count it as a “1”, otherwise as a “0”. Sum up the list of 1’s and 0’s and divide it by 1,000. This represents the likelihood of getting data as or more extreme than what was observed under the null hypothesis (sound familiar?).
* What is your estimated probability?
* Repeat the analysis on the trucks with length 37+.

In [8]:
print sum([trucksim(4989, d).count('35-37') >= 368 for i in range(1000)]) / float(1000) #35-37
print sum([trucksim(4989, d).count('37+') >= 108 for i in range(1000)]) / float(1000) #37+

0.156
0.212


The estimated probabilities of 35-37 and 35+ are 0.156 and 0.212, respectively. This means even though the XXX's algorithm is the same as the current one, the likelihood of getting data as or more extreme than what was observed for 35-37 or for 37+ is quite big.

## Q5
Repeat the same procedure, but this time doing a joint test of both hypothesis (having more extreme values for BOTH 37+ and 35-37). What is the relationship between the tests?

In [9]:
pos = 0
for i in range(1000):
    simlist = trucksim(4989, d)
    accu1 = sum([x == '35-37' for x in simlist])
    accu2 = sum([x == '37+' for x in simlist])
    if accu1 >= 368 and  accu2 >= 108:
        pos += 1
float(pos) / 1000

0.03

The estimated probability of the joint test is 0.03. This means if the XXX's algorithm is not better than the current one, the likelihood of getting data as or more extreme than what was observed for both 35-37 and 37+ is only 0.03, which is quite unlikely. Therefore, we have enough evidence to reject the null hypothesis and could come to the conclusion that the XXX's algorithm is better than the current one.

## Q6
What are the strengths and weaknesses of this method? What do you conclude about XXX’s algorithm?
  
**Bootstrap**
  
Strength: We are able to perform the simulation when we don't have a ton of data and don't know the underlying distribution.
  
Weakness: The sample we used to bootstrap from might not be representative of population, which might lead to inaccurate result.

**Monte-Carlo**
  
Strength: We can estimate a statistic with process certainty when direct calculating is difficult or impossible.
  
Weakness: we need to know the underlying distribution.
  
Since the joint test has a type I error of 0.05, and the sum of type I error of two individual tests is larger than 0.05, so we should trust the joint test. We conclude that the XXX's algorithm is better than the current one from the analysis above.